# Aula 01 - Gravação de audio com Python e JavaScript
## Não esta funcionando a API do chatgpt, então realizei com a API da google mesmo... 


In [ ]:
language = 'pt'

In [ ]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time  => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({audio: true})
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
    }
    recorder.stop()
})
"""

def record(sec=5):
    display(Javascript(RECORD)) # carrega o codigo javascript no colab
    js_result = output.eval_js('record(%s)' % (sec * 1000)) # eval_js = executa o código js (promisse)
    audio = b64decode(js_result.split(',')[1]) # pega o resultado do js e decodifica
    nome_arquivo = 'request_audio.wav'
    with open(nome_arquivo, 'wb') as arquivo: # wb = web binario, salva o arquivo em binario
        arquivo.write(audio)  # escreve o audio em um arquivo
    return f'/content/{nome_arquivo}'


print("estou ouvindo ")
record_file = record() # grava o audio e retorna o nome do arquivo
display(Audio(record_file, autoplay=True))

# Aula 02 - Reconhecimento de fala com o Whisper (Open IA)
## Speech-to-text = google

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
import whisper

model = whisper.load_model("small") # escolhe o tamanho do modelo, quanto maior (mais pesado) tiny, small, large.
resultado = model.transcribe(record_file, fp16=False, language=language) # transcreve o audio gravado em texto
transcription = resultado['text'] # pega o texto do resultado
print(transcription) # imprime o texto

# Aula 03 - Integração com a API do ChatGPT

In [ ]:
!pip install openai

In [ ]:
import openai

openai.api_key = ""

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages=[{"role": "user", "content": transcription}] # envia o texto para o gpt3
)

# choices[0] é a primeira resposta mais assertiva
chatgpt_response = response.choices[0].message.content # pega a resposta do gpt3
print(chatgpt_response) # imprime a resposta

# Aula 04 - Sintetizando voz usando gTTS 
## Text-to-Speech da google

In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS 

gtts_object = gTTS(text=transcription, lang=language, slow=False) # cria um objeto gtts(converte o texto em audio) com a resposta do gpt3 
response_audio = '/content/response_audio.wav' # nome do arquivo de resposta 
gtts_object.save(response_audio) # salva o audio da resposta 
display(Audio(response_audio, autoplay=True)) # exibe o audio da resposta
